# Identifiering av potientiellt igenvuxna stränder kring Vänern med satellitdata <br><br>![title](bilder/metria_logo.jpg)

Verktyget demonstrerar metoder för att identifiera potentiellt igenvuxna<br> stränder kring Vänern med hjälp av Rymddatalabbet och <br>maskininlärningsalgoritmen "Random Forest".<br>https://en.wikipedia.org/wiki/Random_forest
<br><br>Förvalda satellitbilder från Sentinel-2 används, med möjlighet för användaren<br>
att anpassa anpassa parametrar i analysen. Framtida versioner av verktyget <br>kan förbättras genom att integrera äldre satellitdata från till exempel
<br>Landsat eller SPOT när dessa finns tillgängliga i Rymddatalabbet.<br><br>Observera att separat login till Rymddatalabbet krävs för att använda verktyget som tänkt.<br><br>Verktyget är skrivet av [Metria AB](http://www.metria.se) på uppdrag av [Länsstyrelsen i Västra Götaland.](https://www.lansstyrelsen.se/vastra-gotaland.html)

----

<b>Vi börjar med att importera alla nödvändiga moduler:</b>

In [ ]:
import datacube
from utils.data_cube_utilities.dc_display_map import display_map
import rasterio
from rasterio.features import shapes
import fiona
import geopandas as gp
import numpy
import xarray
from matplotlib import pyplot as plt
from IPython.display import Markdown, display
import ipywidgets
from ipywidgets import Layout
from utils.metria_utilities import metria_utils
from sklearn.ensemble import RandomForestClassifier

<br><b>Nedan väljer vi parametrar för vår analys:</b>

* Random Forest-metoden bygger på att ett antal möjliga klassningar testas <br>för att hitta den bästa lösningen. <b>Antal träd</b> avgör mängden försök <br>till klassning som utförs. Fler träd medför längre bearbetningstider. <br>Som standard används 100 träd, men upp till 500 kan användas.<br><br>
* För att kunna hitta igenvuxna ytor krävs referensdata som talar om för <br>Rymddatalabbet hur dessa ytor kan se ut. Ett referensdataset är <br>inkluderat i detta verktyg. <b>Viktning</b> bestämmer den relativa betydelse <br>som pixlar som redan identifierats som igenvuxna av Metria får i klassningen.<br>Som standard används vikten 10, men användaren kan anpassa<br> denna mellan 1-20.
<br><br>
* Analysen begränsas till enskilda <b>kommuner</b> för att garantera att tillräckligt <br>många referensytor finns tillgängliga när verktyget körs.

In [ ]:
kommuner = ['Grums','Grästorp','Gullspång','Götene','Hammarö',
     'Karlstad','Kristinehamn','Lidköping','Mariestad',
     'Mellerud','Säffle','Vänersborg','Åmål']

kommunval = ipywidgets.RadioButtons(
        options=kommuner,
        value=None,
        description='Kommun: ',
        disabled=False,
    )

display(kommunval)

trad = ipywidgets.IntSlider(
    value=100,
    min=100,
    max=500,
    step=1,
    description='Antal träd:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(trad)

vikt = ipywidgets.IntSlider(
    value=10,
    min=1,
    max=20,
    step=1,
    description='Viktning:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

display(vikt)


<b>När cellen nedan körs sparas våra val och aktuell avgränsning visas på karta:</b>

In [ ]:
kommun = kommunval.value

analysyta = {}
with fiona.open('filer/strandlinje_per_kommun.gpkg', 'r', encoding='utf-8') as strandlinje_per_kommun:
    for k in strandlinje_per_kommun:
        if kommun in k['properties']['KOMMUNNAMN'].capitalize():
            bounds = fiona.bounds(k)
            bounds = tuple(map(lambda x: isinstance(x, float) and round(x, -1) or x, bounds))

            min_x = bounds[0]-1000
            min_y = bounds[1]-1000
            max_x = bounds[2]+1000
            max_y = bounds[3]+1000

            bounds = (min_x, min_y, max_x, max_y)

            analysyta[k['properties']['KOMMUNNAMN'].capitalize()] = bounds

            min_ll = metria_utils.omvandla(3006, 4236, bounds[0], bounds[1])[0:2]
            max_ll = metria_utils.omvandla(3006, 4236, bounds[2], bounds[3])[0:2]

            long = (min_ll[0], max_ll[0])
            lat = (min_ll[1], max_ll[1])
            
res = (-10, 10)

opt = {'utan_s':' kommun', 'med_s':'s kommun'}
if kommun.endswith('s') or kommun.endswith('e') or kommun.endswith('ö'):
    kommunstr = kommun+opt['utan_s']
else:
    kommunstr = kommun+opt['med_s']

print('\nSammanfattning av val:\n')
print(f'Analysområde: {kommunstr}\nAntal träd: {str(trad.value)}\nViktning: {vikt.value}\n')

display_map(latitude = lat, longitude = long)

<br><b>Vi importerar referensdata för aktuell kommun och visualiserar på skärm.</b><br><br>Referensdata är baserat på två källor:<br>
1. Modifierade marktäckeklasser från Nationella Marktäckedata ([NMD](https://www.naturvardsverket.se/Sa-mar-miljon/Kartor/Nationella-Marktackedata-NMD/))
2. Igenväxande ytor karterade av Metria.

In [ ]:
ref_data, profile = metria_utils.raster_subset('filer/referensdata_vanern.tif', bounds, res)

klasser = {1:'Igenvuxen yta',
         2: 'Öppen våtmark',
         3: 'Åkermark',
         41: 'Bar öppen mark',
         42: 'Vegeterad öppen mark',
         50: 'Bebyggelse',
         60: 'Vatten',
         100: 'Barrskog',
         101: 'Lövskog',
         118: 'Hygge'}

n_provpunkter = (ref_data > 0).sum()
print(f'\nVi har totalt {n_provpunkter} provpunkter.')

labels = numpy.unique(ref_data[ref_data > 0])
print(f'\nTräningsdatasetet innehåller följande {labels.size} klasser:\n')
print('{' + "\n".join("{!r}: {!r},".format(k, v) for k, v in klasser.items()) + '}\n')
plt.imshow(ref_data, cmap='plasma')

<br><b>När cellen nedan körs hämtas satellitdata för aktuell kommun från Rymddatalabbet.</b><br><br>
Datumen som används är i detta verktyg förvalda för att garantera molnfira satellitbilder:

* 2018-05-30
* 2018-06-29
* 2018-07-14

In [ ]:
satindex = {'s2b_maj':'2018-05-30', 's2b_jun':'2018-06-29', 's2a_jul':'2018-07-14'}

bandlista = ['B02_10m', 'B03_10m','B04_10m', 
             'B05_20m', 'B06_20m', 'B07_20m', 
             'B08_10m', 'B8A_20m', 'B11_20m', 'B12_20m']

dc = datacube.Datacube()

print('Letar efter tillgängliga satellitbilder.')

datasets = []
for satellit, datum in satindex.items():
    if 's2a' in satellit:
        prod = 's2a_sen2cor_granule'
    elif 's2b' in satellit:
        prod = 's2b_sen2cor_granule'

    kriterier = {
            'x': (bounds[0], bounds[2]),
            'y': (bounds[1], bounds[3]),
            'crs': 'EPSG:3006',
        
            'output_crs': 'EPSG:3006',
            'resolution': (-10, 10),
            'time': (datum)
                }
    try:
        dataset = dc.load(product=prod,
                          group_by='solar_day',
                          **kriterier)        
        datasets.append(dataset)
    except ValueError:
        pass    

try:
    satellitdata = xarray.concat(datasets, dim='time').sortby('time')
    band = satellitdata[bandlista]    
except Exception:
    band = dataset[bandlista]

final_array = []
for b in bandlista:
    for array in band[b]:
        final_array.append(array.values)
stack = numpy.asarray(final_array)
stack = numpy.moveaxis(stack, 0, 2)
        
print(f'\nAnvänder {len(band.time)} bildtillfällen för analysen:\n')

rgbset = satellitdata[["B04_20m", "B03_20m", "B02_20m"]]
rgbset.to_array().plot.imshow(col = "time", col_wrap = 3, robust = True)

<br><b>Nedan testas våra satellitdata mot våra referensdata för att skapa en klassningsmodell (detta kan ta ett tag)</b><br><br>
Verktyget jämför klasserna i referensdata med information från satellitbilderna i Rymddatalabbet för att identifiera <br>ytor som liknar referensdata, däribland igenväxande ytor.<br>

När detta steg är färdigt presenteras en uppskattad klassningsnoggrannhet samt en lista som indikerar hur viktiga enskilda<br> band är för att genomföra klassificeringen:

In [ ]:
sat_filter = stack[ref_data > 0]
ref_filter = ref_data[ref_data > 0]

l = 1

random_forest = RandomForestClassifier(n_estimators=trad.value, oob_score=True, class_weight={
        1: vikt.value, 2:l, 3:l, 41:l, 42:l, 50:l, 60:l, 100:l, 101:l, 118:l})
rf = random_forest.fit(sat_filter, ref_filter)

print(f'Vår uppskattade klassningsnoggrannhet är: {rf.oob_score_ * 100}\n')
bands = list(range(1, stack.shape[2]+1))

for b, imp in zip(bands, rf.feature_importances_):
    print(f'Relativ betydelse band {b}: {imp}')

<br><b>När klassningsmodellen är färdig använder vi den för att generera ett resultat</b><br><br>
Analysen genererar först en rasterfil som består av alla pixlar som klassificerats som igenvuxen yta. Dessa konverteras till vektor-polygoner, <br>och sedan tas centroid-punkten ut från alla polygoner. Det resulterande punkt-skiktet visas på skärm och kan sparas som graf, eller en shapefil i nästa steg. <br>Punkterna ger en indikation på var verktyget tror sig ha hittat en igenväxande yta:

In [ ]:
template = (stack.shape[0] * stack.shape[1], stack.shape[2])
ny_stack = stack[:].reshape(template)
klassning = rf.predict(ny_stack)
klassning = klassning.reshape(stack[:, :, 0].shape)
resultat = numpy.copy(klassning)

strand, profile = metria_utils.raster_subset('filer/vanern_strandzon_200m.tif', bounds, res)

resultat[resultat > 1] = 0
resultat[strand == 0] = 0

pg = (
    {'properties': {'raster_val': v}, 'geometry': s}
    for i, (s, v) 
    in enumerate(
        shapes(resultat, mask=None, transform=profile['transform'])))

geoms = list(pg)
geoms = [x for x in geoms if not x['properties']['raster_val'] == 0]

polygoner  = gp.GeoDataFrame.from_features(geoms)
centroider = polygoner['geometry'].centroid

print(f'\nHittade totalt {len(centroider)} punkter med potentiell igenväxning i {kommunstr}.\n')

strand = gp.read_file('filer/strandlinje_per_kommun.gpkg')

base = strand.plot(edgecolor='black', figsize=(12, 12))
centroider.plot(ax=base, color='red')
base.set_title(f"Potentiellt igenvuxna områden i {kommunstr}\n", fontsize=20)
base.set_xlim(bounds[0], bounds[2])
base.set_ylim(bounds[1], bounds[3])

knapp = ipywidgets.Button(
    description='Klicka här för att spara grafen nedan som en bild',
    layout=Layout(width='40%'),
    button_style='',
    tooltip='Klicka här för att spara grafen nedan som en bild',
    icon='check')
out = ipywidgets.Output()

def knapp_click(_):
      with out:
            base = strand.plot(edgecolor='black', figsize=(12, 12))
            centroider.plot(ax=base, color='red')
            base.set_title(f'Potentiellt igenvuxna områden i {kommunstr}\n', fontsize=20)
            base.set_xlim(bounds[0], bounds[2])
            base.set_ylim(bounds[1], bounds[3])
            plt.savefig(f'resultat/Potentiellt igenvuxna områden i {kommunstr}.png'.replace(' ', '_'))
            print('Din fil är sparad under "resultat/"')

knapp.on_click(knapp_click)
ipywidgets.VBox([knapp,out])

<br><b>Om vi är nöjda med analysen kan vi spara resultatet som en shapefil för att använda i ett GIS (hamnar i foldern "resultat" till vänster):</b>

In [ ]:
filnamn = ipywidgets.Text(
    value=f'Potentiell_igenväxning_i_{kommunstr}.shp'.replace(' ', '_'),
    layout=Layout(width='70%'),
    description='Filnamn: ',
    disabled=False
)

display(filnamn)
print('\n')

knapp = ipywidgets.Button(
    description='Klicka här för att spara resultatet som en shapefil',
    layout=Layout(width='40%'),
    button_style='',
    tooltip='Klicka här för att spara resultatet som en shapefil',
    icon='check')
out = ipywidgets.Output()

def knapp_click(_):
      with out:
            centroider.to_file(f'resultat/{filnamn.value}')
            print('Din fil är sparad under "resultat/"')

knapp.on_click(knapp_click)
ipywidgets.VBox([knapp,out])